In [1]:
# List file data
import glob

path = 'data/Data/'

files = [f[10:] for f in glob.glob(path + "*.txt", recursive=True)]

for f in files:
    print(f)


data_100_1.txt
data_100_10.txt
data_100_2.txt
data_100_5.txt
data_120_1.txt
data_120_10.txt
data_120_2.txt
data_120_5.txt
data_150_1.txt
data_150_10.txt
data_150_2.txt
data_150_5.txt
data_200_1.txt
data_200_2.txt
data_200_5.txt
data_50_10.txt
data_50_100.txt
data_50_2.txt
data_50_5.txt
data_50_50.txt
data_80_10.txt
data_80_2.txt
data_80_5.txt


In [2]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Sensor:
    def __init__(self, index, x, y, lr):
        self.index = index
        self.circle = Point(x, y)
        self.lr = lr
    
    @property
    def index(self):
        return self.__index
    
    @index.setter
    def index(self, index):
        self.__index = index

    @property
    def circle(self):
        return self.__circle
    
    @circle.setter
    def circle(self, circle):
        self.__circle = circle
    
    @property
    def lr(self):
        return self.__lr
    
    @lr.setter
    def lr(self, lr):
        self.__lr = lr

Sensor.s = Sensor(0, 0, 0, 0)
Sensor.t = Sensor(1, 0, 0, 0)


In [3]:
class Distance:
    @staticmethod
    def distancePoint(x1, y1, x2, y2):
        return np.sqrt((x2-x1)*(x2-x1)+(y2-y1)*(y2-y1))

    @staticmethod
    def distance(x1, y1, r1, x2, y2, r2):
        r = Distance.distancePoint(x1, y1, x2, y2)
        if (r <= r1 + r2):
            return 0
        return r - r1 - r2
    
    @staticmethod
    def distancew(x1, r1, x2, r2):
        r = np.abs(x1-x2)
        if (r <= r1 + r2):
            return 0
        return r - r1 - r2

    @staticmethod
    def dw(vi, vj, l, lr):
        # trung
        if (vi == vj):
            return 0
        if (vi == Sensor.s and vj == Sensor.t or vi == Sensor.t and vj == Sensor.s):
            return int(np.ceil (l/lr))
        if (vi == Sensor.s):
            return int(np.ceil(Distance.distancew(0, 0, vj.circle.x, vj.lr)/lr))
        if (vi == Sensor.t):
            return int(np.ceil(Distance.distancew(l, 0, vj.circle.x, vj.lr)/lr))
        if (vj == Sensor.s):
            return int(np.ceil(Distance.distancew(vi.circle.x, vi.lr, 0, 0)/lr))
        if (vj == Sensor.t):
            return int(np.ceil(Distance.distancew(vi.circle.x, vi.lr, l, 0)/lr))
        return int(np.ceil(Distance.distancew(vi.circle.x, vi.lr, vj.circle.x, vj.lr)/lr))
    
    @staticmethod
    def ds(vi, vj, l, lr):
        # trung
        if (vi == vj):
            return 0
        # Khoang cach tu dinh ao s, t theo cong thuc weak barrier
        if (vi == Sensor.s or vi == Sensor.t or vj == Sensor.s or vj == Sensor.t):
            return Distance.dw(vi, vj, l, lr)
        # Neu 2 sensor la sensor thuong
        return int(np.ceil(Distance.distance(vi.circle.x, vi.circle.y, vi.lr, vj.circle.x, vj.circle.y, vj.lr)/lr))

    @staticmethod
    def w(vi, vj, l, lr, weak):
        if (weak):
            return Distance.dw(vi, vj, l, lr)
        return Distance.ds(vi, vj, l, lr)


In [4]:
import numpy as np

class Data:
    def __init__(self, L, H, k, lr, listSensor, weak = False):
        self.L = L
        self.H = H
        self.k = k
        self.lr = lr
        self.listSensor = listSensor
        self.weak = weak
        self.wbg = WBG(self)
    
    @property
    def L(self):
        return self.__L
    
    @L.setter
    def L(self, L):
        self.__L = L
    
    @property
    def H(self):
        return self.__H
    
    @H.setter
    def H(self, H):
        self.__H = H

    @property
    def k(self):
        return self.__k
    
    @k.setter
    def k(self, k):
        self.__k = k

    @property
    def lr(self):
        return self.__lr
    
    @lr.setter
    def lr(self, lr):
        self.__lr = lr

    @property
    def weak(self):
        return self.__weak
    
    @weak.setter
    def weak(self, weak):
        self.__weak = weak

    @property
    def listSensor(self):
        return self.__listSensor
    
    @listSensor.setter
    def listSensor(self, listSensor):
        self.__listSensor = listSensor
    
    @property
    def wbg(self):
        return self.__wbg
    
    @wbg.setter
    def wbg(self, wbg):
        self.__wbg = wbg

    @staticmethod
    def readData(filename):
        with open(filename, 'r') as file:
            split = file.readline().split(' ')
            L, H, k, lr = float(split[0]), float(split[1]), int(split[2]), float(split[3])
            n = int(file.readline())
            listSensor = []
            for i in range(0, n):
                x, y, a, lr = [0 if f == '' else float(f) for f in file.readline().split(' ')]
                s = Sensor(i+2, x, y, lr)
                listSensor.append(s)
            file.close()
        return Data(L, H, k, lr, listSensor)

class Node:
    def __init__(self, index, s):
        self.index = index
        self.s = s

class WBG:    
    def __init__(self, data):
        self.d = data
        self.__buildWBG();
        
        # Print WBG
        # print("WBG: ")
        #for i in range(0, len(self.wbg)):
        #    print("[", end = '')
        #    for j in range(0, len(self.wbg)):
        #        print("%2d," % self.wbg[i][j], end = ' ')
        #    print("]")

    def __buildWBG(self):
        # Size of wbg
        self.size = len(self.d.listSensor)+2

        # List of node
        self.nodes = []
        
        # Build node
        for i in range(0, self.size):
            self.nodes.append(Node(i, Sensor.s if i == 0 else Sensor.t if i == 1 else self.d.listSensor[i-2]))

        # The wbg
        self.wbg = np.full((self.size, self.size), 0)

        #Build wbg
        for i in range(0, self.size):
            for j in range(0, self.size):
                self.wbg[j][i] = self.wbg[i][j] = Distance.w(self.nodes[i].s, self.nodes[j].s, self.d.L, self.d.lr, self.d.weak)

    @property
    def size(self):
        return self.__size
    
    @size.setter
    def size(self, size):
        self.__size = size

    @property
    def wbg(self):
        return self.__wbg
    
    @wbg.setter
    def wbg(self, wbg):
        self.__wbg = wbg

    @property
    def nodes(self):
        return self.__nodes
    
    @nodes.setter
    def nodes(self, nodes):
        self.__nodes = nodes


In [ ]:
d = Data.readData('data/Data/data_100_50.txt')

print(d.L, ' ', d.H, ' ', d.k, ' ', d.lr)
print(len(d.listSensor))

from mip import Model, xsum, minimize, BINARY
import time

start = time.process_time()

size = d.wbg.size
I = range(size)
K = range(2, size)
J = range(d.k)

m = Model()

x = [[[m.add_var(var_type=BINARY) for k in J] for j in I] for i in I]

m.objective = minimize(xsum(d.wbg.wbg[i][j] * x[i][j][k] for i in I for j in I for k in J))

for l in J:
    m += xsum(x[0][j][l] for j in I)-xsum(x[j][0][l] for j in I) == 1
    m += xsum(x[1][j][l] for j in I)-xsum(x[j][1][l] for j in I) == -1

    for i in K:
        m += xsum(x[i][j][l] for j in I)-xsum(x[j][i][l] for j in I) == 0

for i in K:
    m += xsum(x[i][j][l] for j in I for l in J)+xsum(x[j][i][l] for j in I for l in J) <= 2

m.optimize()

allW = 0
for l in J:
    sensors = [0]
    k = 0
    w = 0
    while k != 1:
        for j in I:
            if x[k][j][l].x >= 0.99:
                sensors.append(j)
                w += d.wbg.wbg[k][j]
                k = j
                break
    allW += w
    print (l, ": [", end = '')
    for i in range(0, len(sensors)-1):
        print (sensors[i], ", ", end = '')
    print (sensors[-1], ']', ' wbg=', w)

print ()
print ("Total: ", allW)
print ("Time: ", time.process_time() - start, "s")


In [ ]:
from mip import Model, xsum, minimize, BINARY

import time

for f in files:
    d = Data.readData(f)
    
    start = time.process_time()
    
    size = d.wbg.size
    I = range(size)
    K = range(2, size)
    J = range(d.k)

    m = Model()

    x = [[[m.add_var(var_type=BINARY) for k in J] for j in I] for i in I]

    m.objective = minimize(xsum(d.wbg.wbg[i][j] * x[i][j][k] for i in I for j in I for k in J))

    for l in J:
        m += xsum(x[0][j][l] for j in I)-xsum(x[j][0][l] for j in I) == 1
        m += xsum(x[1][j][l] for j in I)-xsum(x[j][1][l] for j in I) == -1

        for i in K:
            m += xsum(x[i][j][l] for j in I)-xsum(x[j][i][l] for j in I) == 0

    for i in K:
        m += xsum(x[i][j][l] for j in I for l in J)+xsum(x[j][i][l] for j in I for l in J) <= 2

    m.optimize()

    allW = 0
    for l in J:
        for i in I:
            for j in I:
                if x[i][j][l].x >= 0.99:
                    allW += d.wbg.wbg[i][j]

    print (f, ": Total=", allW, "Time=", time.process_time() - start, "s")


In [41]:
import os
import time

currentTimeMillis = lambda: int(round(time.time() * 1000))

class Algorithm:
    def __init__(self, name):
        self.name = name
    
    def doAlgorithm(self):
        pass
    
    @property
    def time(self):
        return self.__time

    @time.setter
    def time(self, time):
        self.__time = time

    @property
    def nm(self):
        return self.__nm
    
    @nm.setter
    def nm(self, nm):
        self.__nm = nm
    
    @property
    def barriers(self):
        return self.__barriers
    
    @barriers.setter
    def barriers(self, barriers):
        self.__barriers = barriers
    
    def readOutput(self, filename):
        with open(filename, 'r') as file:
            self.d.k = int(file.readline())
            self.nm = int(file.readline())
            self.time = int(file.readline())
            file.close()

    def solve(self, odir, fname, data):
        self.d = data
        
        filename = odir + self.name + "/" + fname+".txt" # output file path

        # File f = new File(filename.substring(0, filename.lastIndexOf("/"))); // output directory
        # if(!f.exists()) // create if not exists
        #    f.mkdirs();

        if (os.path.exists(filename)):
            self.readOutput(filename) # read the output
            return

        self.time = currentTimeMillis() # start time
        self.doAlgorithm() # do solve
        self.time = currentTimeMillis()-self.time # end time

        with open(filename, 'w') as file:
            # Output
            file.write(str(self.d.k)+"\n")
            file.write(str(self.nm)+"\n")
            file.write(str(self.time)+"\n")

            # Barrier
            for i in range(0, d.k):
                file.write("[")
                b = self.barriers[i]
                for j in range(0, len(b)-1):
                    file.write(str(b[j].index)+",")
                file.write("1]\n")

            file.close()
            
            # draw(odir, fname+".png", data) # ve hinh output

In [42]:
from mip import Model, xsum, minimize, BINARY

class ILP(Algorithm):
    def __init__(self):
        super().__init__("ILP")
    
    def doAlgorithm(self):
        size = self.d.wbg.size
        I = range(size)
        K = range(2, size)
        J = range(d.k)

        m = Model()

        x = [[[m.add_var(var_type=BINARY) for k in J] for j in I] for i in I]

        m.objective = minimize(xsum(d.wbg.wbg[i][j] * x[i][j][k] for i in I for j in I for k in J))

        for l in J:
            m += xsum(x[0][j][l] for j in I)-xsum(x[j][0][l] for j in I) == 1
            m += xsum(x[1][j][l] for j in I)-xsum(x[j][1][l] for j in I) == -1

            for i in K:
                m += xsum(x[i][j][l] for j in I)-xsum(x[j][i][l] for j in I) == 0

        for i in K:
            m += xsum(x[i][j][l] for j in I for l in J)+xsum(x[j][i][l] for j in I for l in J) <= 2

        m.optimize()

        self.nm = 0
        self.barriers = []
        for l in J:
            sensors = [self.d.wbg.nodes[0]]
            k = 0
            while k != 1:
                for j in I:
                    if x[k][j][l].x >= 0.99:
                        sensors.append(self.d.wbg.nodes[j])
                        self.nm += d.wbg.wbg[k][j]
                        k = j
                        break
            self.barriers.append(sensors)


In [43]:
ilp = ILP()

with open("ketqua"+ilp.name+".txt", "w") as file:
    file.write("filename          best      avg      dlc     time \n")

    for name in files:
        print (name)
        d = Data.readData(path+name)
        ilp.solve("./data/Result/", "Data_"+name+"_Run_0", d)
        file.write(name + "      " + str(ilp.nm) + "      " + str(ilp.nm) + "      " + str(0.0) + "      " + str(ilp.time) + "\n")
        file.write("-------------------------------------------------\n")
    
    file.close()


data_100_1.txt
data_100_10.txt
data_100_2.txt
data_100_5.txt
data_120_1.txt
data_120_10.txt
data_120_2.txt
data_120_5.txt
data_150_1.txt
data_150_10.txt
data_150_2.txt
data_150_5.txt
data_200_1.txt
data_200_2.txt
data_200_5.txt
data_50_10.txt
data_50_100.txt
data_50_2.txt
data_50_5.txt
data_50_50.txt
data_80_10.txt
data_80_2.txt
data_80_5.txt
